In [1]:
%env HSA_OVERRIDE_GFX_VERSION=10.3.0
%env ROCM_PATH=/opt/rocm


env: HSA_OVERRIDE_GFX_VERSION=10.3.0
env: ROCM_PATH=/opt/rocm


In [2]:
import subprocess
import os
import pathlib
import configparser
import sqlite3 as sl
import torchaudio
import cv2
from pydub import AudioSegment
import math
import pickle
import shutil
import time
import multiprocessing
from multiprocessing import Process,Queue
import itertools
from threading import Thread
import soundfile as sf
import extractOpenL3Subprocess as openSb
from multiprocessing import Queue
from multiprocessing import Pool


start_time = time.time()    # To measure execution time in seconds

print("PLEASE EDIT configuration.txt BEFORE EXECUTION")
print(".wav files might be generated in path. The program will automatically delete them. If execuetion stops unexpectedly, please delete them yourself")

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)

datasetPathVideo =  configParser.get('COMMON', 'datasetPathVideo')
datasetPathDatabase =  configParser.get('COMMON', 'datasetPathDatabase') + '/dataset.db'
cpus =  int(configParser.get('COMMON', 'cpus'))

datasetPathAudio =  configParser.get('extractOpenL3', 'datasetPathAudio')

p =  configParser.get('extractOpenL3', 'dbChunk')
ttwbdf =  int(configParser.get('extractOpenL3', 'time_to_wait_before_deleting_files'))

print("Video dataset at " + datasetPathVideo )



/home/gamal/anaconda3/envs/ds2f_pre/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-25 19:59:11.766416: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


PLEASE EDIT configuration.txt BEFORE EXECUTION
.wav files might be generated in path. The program will automatically delete them. If execuetion stops unexpectedly, please delete them yourself
Video dataset at /home/gamal/Datasets/Dataset1/Video


In [3]:
con = sl.connect(datasetPathDatabase)  # Connection to databases
print('------------------- ABOUT TO START --------------------')
#TODO what if two files have the same name in the same batch

 
def extractAudio(rows):
    con2 = sl.connect(datasetPathDatabase)
    #print(rows)

    for row in rows:
        #print(row)
        absPathVideo = row[1]   # for this one video
        rowId = row[0]          # id in database

        absPathAudio = y = absPathVideo.replace(datasetPathVideo,datasetPathAudio)  # for this one audio
        absPathAudio = os.path.splitext(absPathAudio)[0]
        absPathAudio_w = absPathAudio   # without the end
        absPathAudio = absPathAudio + "_audio.wav"  # full path to extracted audio from the video

        #Create Directory
        pathlib.Path(os.path.dirname(absPathAudio)).mkdir(parents=True, exist_ok=True) 

        # Extract audio monochannel and with 16khz and put it in absPathAudio
        command = "ffmpeg -nostats -loglevel 0 -y -i '" + absPathVideo + "' -acodec pcm_s16le -ab 160k -ac 1 -ar 16000 -vn '" + absPathAudio + "'"
        subprocess.call(command, shell=True)


        # Get original duration of video
        audio = AudioSegment.from_file(absPathVideo)
        audio_length_og = math.floor(audio.duration_seconds)
        #print(audio_length_og)
        

    
        # Will either truncate or loop the original video to reach audio_length (3,6,12 or 24)
        audio_length_list = [6,12,24]
        for audio_length in audio_length_list:
            path_var_len_audio =  absPathAudio_w + "audio" + str(audio_length) + "s.wav"    # path to the variable length audio
            path_var_len_audio_temp =  absPathAudio_w + "audio_temp" + str(audio_length) + "s.wav"  # path to a temp version of the variable length audio

            if(audio_length_og > audio_length):
                # Truncate    

                command = "ffmpeg -nostats -loglevel 0 -y -ss 0 -t "+str(audio_length)+" -i \"" + absPathAudio + "\" \"" + path_var_len_audio + "\""
                subprocess.call(command, shell=True)


            else:
                # Loop then truncaate
                #print("lesa")
                twoDigitLenStr = f"{audio_length:02}"
                #print(twoDigitLenStr)
                command = "ffmpeg -nostats -loglevel 0 -y -stream_loop -1 -i '" + absPathAudio + "' -t \"00:00:"+twoDigitLenStr+".000\" -codec:a \"aac\" -f \"wav\" -c copy '"+ path_var_len_audio_temp + "'"
                subprocess.call(command, shell=True)
                command = "ffmpeg -nostats -loglevel 0 -y -ss 0 -t "+str(audio_length)+" -i \"" + path_var_len_audio_temp + "\" \"" + path_var_len_audio + "\""
                subprocess.call(command, shell=True)

            # Extract audio embeddings
            audio, sr = sf.read(path_var_len_audio)
            #emb = []
            #queue = Queue()
            #proc = Process(target=openSb.extractOpenL3Subprocess, args=(audio,sr,queue,))   # spawn a process
            #proc.start()
            #proc.join()
            #emb = queue.get()

            p = Pool(1)
            emb = p.apply(openSb.extractOpenL3Subprocess, args=(audio,sr,))

            #emb, ts = openl3.get_audio_embedding(audio, sr,embedding_size=512,hop_size=audio_length/50,verbose=0)
            #signal, fs = torchaudio.load(path_var_len_audio)
            #embeddings = classifier.encode_batch(signal)
            #embeddingsPickle = pickle.dumps(embeddings.cpu().detach().numpy()) # pickle embeddings to put in database
            #print(emb)
            #print(emb.min())
            #print(emb.max())
            #print(audio_length)
            #print(emb.shape)
            #print(emb)
            embeddingsPickle = pickle.dumps(emb)


            # update audio embeddings into database
            sql = ''' UPDATE AUDIO SET AUDIO_EMB = ? WHERE VIDEO_ID = ? AND AUDIO_LENGTH = ?'''

            
            cur = con2.cursor()
            data = [embeddingsPickle,rowId,audio_length]
            cur.execute(sql, data)
            con2.commit()
            
            sql = '''UPDATE VIDEO SET AUDIO_PRE = 2 WHERE ID = ?'''

            data = [rowId]
            cur.execute(sql, data)
            con2.commit()
            cur.close()

             # Will delete those files after a little bit
            ftd = [absPathAudio,path_var_len_audio,os.path.basename(path_var_len_audio),path_var_len_audio_temp]
            tDelete = Thread(target=delFiles, args=(ftd,))   # spawn a process
            tDelete.start()
           
            
# Function to delete audio temp files
def delFiles(filesToDelete):
    time.sleep(ttwbdf)  # wait a bit
    for file in filesToDelete:  
        try:
            os.remove(file)
        except OSError:
            pass
        



# TODO Better display of progress and handling of exceptions
contLoop = True # Flag to continue to get chunks of videos from database
offset = 0
while(contLoop):
    data = con.execute("SELECT * FROM VIDEO WHERE AUDIO_PRE = 1 ORDER BY ID ASC LIMIT " + p + " OFFSET " + str(offset))
    contLoop = False
    offset = offset + int(p)
    print("Got chunk of videos from database. Extracting audio and audio embeddings...")
    # TODO write time
    
    #print(data.fetchall())
    dataGotten = data.fetchall()
    rowsPerProcess = math.ceil(len(dataGotten) / cpus)  # Will spawn no. of processes = cpus, each will get rows = rowsperprocess
    procs = []
    while(len(dataGotten) > 0):
        rows=dataGotten[:rowsPerProcess]    # rows to be sent to a process
        dataGotten = dataGotten[rowsPerProcess:]    # Deletes the rows that are going to be sent from dataGotten
        #print(rows)
        contLoop = True # Continue to get data from database since data length is not 0
        extractAudio(rows)
        #proc = Process(target=extractAudio, args=(rows,))   # spawn a process
        #procs.append(proc)
        #proc.start()
    #for proc in procs:  # wait for all processes to finish
        #proc.join()
    
print('----------------------------------------------------------------      FINISHED      -----------------------------------------')

print("--- %s seconds ---" % (time.time() - start_time))

------------------- ABOUT TO START --------------------
Got chunk of videos from database. Extracting audio and audio embeddings...


2023-05-25 19:59:13.245722: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 19:59:13.246759: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Extracted OpenL3 features
[[2.2157934 1.3539672 2.3574586 ... 1.3633066 2.2015047 4.711986 ]
 [1.7192707 1.568412  1.6576047 ... 1.3956017 3.4050884 2.9535542]
 [1.7817397 1.5138012 1.3830465 ... 1.2993425 3.37405   3.2152371]
 ...
 [1.3433698 1.5630363 2.7006621 ... 1.6842291 2.6544116 2.8875177]
 [1.2797408 1.5736521 2.5085328 ... 1.643735  2.7840748 2.9006176]
 [1.8949546 1.47123   2.987916  ... 1.3454193 2.1860504 6.0763574]]


2023-05-25 19:59:15.108341: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 19:59:15.109201: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Extracted OpenL3 features
[[2.2157934 1.3539672 2.3574586 ... 1.3633066 2.2015047 4.711986 ]
 [1.7192707 1.568412  1.6576047 ... 1.3956017 3.4050884 2.9535542]
 [1.7817397 1.5138012 1.3830465 ... 1.2993425 3.37405   3.2152371]
 ...
 [1.7870079 1.5772071 1.6725558 ... 1.4061599 3.423778  2.9732575]
 [1.7287073 1.5231851 1.155715  ... 1.2806267 2.783021  2.415559 ]
 [1.3951778 1.5001312 2.6599278 ... 1.6866424 2.650175  2.8785033]]


...

Extracted OpenL3 features
[[2.2157934 1.5402632 2.3574586 ... 1.7413275 1.8087693 4.711986 ]
 [1.5531752 2.0521145 1.9746509 ... 1.6781694 2.1313488 3.2681732]
 [2.1746356 1.9143783 2.0345585 ... 1.6767113 2.1651633 3.3942106]
 ...
 [1.5455052 1.4908237 1.936583  ... 1.3055086 3.4817226 2.7008631]
 [1.5536499 1.3695844 1.4776359 ... 1.1680614 4.2815638 3.4112995]
 [2.3280833 1.408306  3.0631824 ... 1.1741109 4.157827  6.2542396]]


2023-05-25 20:10:19.716284: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 20:10:19.717132: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Extracted OpenL3 features
[[2.2157934 1.5402632 2.3574586 ... 1.7413275 1.8087693 4.711986 ]
 [1.5531752 2.0521145 1.9746509 ... 1.6781694 2.1313488 3.2681732]
 [2.1746356 1.9143783 2.0345585 ... 1.6767113 2.1651633 3.3942106]
 ...
 [1.1869344 1.8297826 1.3073981 ... 1.6648184 3.3415165 2.461196 ]
 [1.3863348 1.478938  2.4219983 ... 1.695995  2.6055543 3.2047024]
 [1.5217855 1.8216913 2.3750677 ... 1.4266486 2.7438672 3.2531283]]


2023-05-25 20:10:22.171420: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 20:10:22.172125: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Extracted OpenL3 features
[[2.2157934 1.5402632 2.3574586 ... 1.7413275 1.8087693 4.711986 ]
 [1.5531752 2.0521145 1.9746509 ... 1.6781694 2.1313488 3.2681732]
 [2.1746356 1.9143783 2.0345585 ... 1.6767113 2.1651633 3.3942106]
 ...
 [1.819444  1.4429334 1.2939427 ... 1.842958  3.0354388 2.8626566]
 [2.039007  1.6327671 2.7516332 ... 1.8132998 3.4831538 2.9995732]
 [2.0055168 1.6402584 2.7719233 ... 1.5938463 3.606671  3.123549 ]]
Got chunk of videos from database. Extracting audio and audio embeddings...
----------------------------------------------------------------      FINISHED      -----------------------------------------
--- 673.2496371269226 seconds ---
